In [1]:
from os import listdir
from os.path import isfile, join
import subprocess
onlyfiles = [f for f in listdir("Generated_descriptions/") if isfile(join("Generated_descriptions/", f))]

In [2]:
onlyfiles

['Pat Simmons.txt',
 'Larsen syndrome.txt',
 'Calmurid.txt',
 'TOLLIP.txt',
 'Anthranilic acid.txt',
 'Vanillyl mandelic acid.txt',
 'SLC7A9.txt',
 'SOX4.txt',
 'Amatoxin.txt',
 'Theria.txt']

In [3]:
content = []
for file in onlyfiles:
    with open("Generated_descriptions/"+file) as f:
        content += [(f.read(), file)]

In [4]:
len(content)

10

In [5]:
def remove_newlines(text):
    return text.replace('\n', ' ')

def save_to_file(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

In [11]:
def run_mistral_with_input(input_text, timeout):
    process = subprocess.Popen(['ollama', 'run', 'mistral'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        stdout, stderr = process.communicate(input_text, timeout=timeout)
        
        if process.returncode != 0:
            print(f"Error executing command: {stderr}")
            return None

    except subprocess.TimeoutExpired:
        process.terminate()
        process.wait()
        print("Process timed out and was terminated.")
        return None

    finally:
        if process.poll() is None: 
            process.terminate()
        process.wait()
    return stdout


In [16]:
for (text,file) in content:
    input_text = " Here is a patient description : " +text+ """
Fill the following structure accordingly. Don't change it.
If the information is not given, don't write anything, leave it as an empty section.
If an information is true for multiple symptoms, repeat the information.
Only answer with the filled structure.

{
  "symptoms": [
    {
      "name of the symptom": "",
      "intensity of symptom": "",
      "specific attributes of the symptom": {
        "location": "",
        "size": "",
        "color": "",
        "frequency": ""
      },
      "When did the symptom appear ": "",
      "previous treatments": "",
      "reaction to previous treaments"
      "behaviour affecting the symptom": ""
    }
  ],
  "socio economic context": {
  },
  "geographic_context": {
      "recent travels": "",
      "level of care": ""
  },
  "physiological context": {
  
  },
  "psychological context": {
  
  },
  "personal medical_history": {
  
  },
  "family medical_history": {
  
  },
  "current medication": {

  },
  "lifestyle factors": {
  
  },
  "results of recent medical tests": {
  
  }
}
<END>
"""

    input_text = remove_newlines(input_text)
    
    response = run_mistral_with_input(input_text, 60 * 5)
    if response:
        with open("output/"+file[:-4], "w") as file:
            file.write(response)
        print("Mistral's response saved to response.txt")

Mistral's response saved to response.txt
Process timed out and was terminated.


KeyboardInterrupt: 

In [14]:
run_mistral_with_input("Salut ?", timeout = 60)

'>>> Salut ?\nSalut! Comment allez-vous ?\n\n'